## P2 - Competition assay and competitive exclusion in serial transfers
Competition experiments are frequently performed in the laboratory to assay, for example, the fitness of a mutant in competition to the wild-type. Here, we simulate one such experiemnt involving E. coli and a nonessential but deleterious mutation involving the deletion of the triose phosphate isomerase reaction from glycolisis.  

We start as usual by creating a layout ("world") and setting the initial nutrient supply.

In [1]:
# Start by loading required packages, including the COMETS toolbox
import comets as c
import pandas as pd

# create an empty layout
world = c.layout()

# Add glucose and remove o2 
world.set_specific_metabolite('glc__D_e', 0.01)

building empty layout model
models will need to be added with layout.add_model()


We will now add the typical trace metabolites to the layout we generated, including oxygen. These metabolites will be considered as "static" by the simulation, with a value of 1000 that never depletes. Considering metabolites as "static" is the way COMETS has to simulate an unlimited supply of a metabolite.  

In [2]:
# Add typical trace metabolites and oxygen coli as static
trace_metabolites = ['ca2_e',
                     'cl_e',
                     'cobalt2_e',
                     'cu2_e',
                     'fe2_e',
                     'fe3_e',
                     'h_e',
                     'k_e',
                     'h2o_e',
                     'mg2_e',
                     'mn2_e',
                     'mobd_e',
                     'na1_e',
                     'ni2_e',
                     'nh4_e',
                     'o2_e',
                     'pi_e',
                     'so4_e',
                     'zn2_e']
        
for met in trace_metabolites:
    newrow = {'metabolite': met,
              'g_refresh': 0,
              'g_static': 1,
              'g_static_val': 1000,
              'init_amount': 1000,
              'diff_c': world.default_diff_c}

    newrow = pd.DataFrame([newrow], columns=newrow.keys())
    world.media = pd.concat([world.media, newrow], axis=0, sort=False) 


We then load our model and create the mutant by setting both upper and lower bounds to zero. We will add both models to our "world". 

In [3]:
# load the models and perform the mutation
wt = c.model('test_models/iJO1366.xml')
mut = c.model('test_models/iJO1366.xml')
mut.change_bounds('TPI', 0,0)
mut.id = 'TPI_iJO1366'

# set its initial biomass, 5e-6 gr at coordinate [0,0]
wt.initial_pop = [0, 0, 5e-8]
mut.initial_pop = [0, 0, 5e-8]

# add it to the world
world.add_model(wt)
world.add_model(mut)

Academic license - for non-commercial use only


We next create a parameters object and modify needed parameters - in this case only the number of cycles the simulation runs.  

In [4]:
comp_params = c.params()
comp_params.all_params['maxCycles'] = 240

Finally, we create the comets object using the above created layout and parameters, and run the competition assay. 

In [5]:
comp_assay = c.comets(world, comp_params)
comp_assay.run()


Running COMETS simulation ...
Done!


We now plot the biomasses of these two genotypes in coculture. 

In [6]:
biomass = comp_assay.total_biomass
biomass['t'] = biomass['cycle'] * comp_assay.parameters.all_params['timeStep']

import matplotlib.pyplot as plt
myplot = biomass.drop(columns=['cycle']).plot(x = 't')

Now we can compute the competitive fitness of the mutant respect to the wild-type as the ratio of the biomass increase of the mutant divided by that of the wild-type: 

In [7]:
cfit = (biomass.loc[biomass['t'] == 24, 'TPI_iJO1366'].iloc[0]/biomass.loc[biomass['t'] == 0, 'TPI_iJO1366'].iloc[0])/(biomass.loc[biomass['t'] == 24, 'iJO1366'].iloc[0]/biomass.loc[biomass['t'] == 0, 'iJO1366'].iloc[0])
cfit

0.3937873212008413

Lets simulate a serial tranfer competition between these two mutants. We just create a different set of parameters, increasing the simulation time and including batch transfers of 1% every 24h, but we will use the same "world" layout as before. 

In [8]:
serial_params = c.params()
serial_params.all_params['maxCycles'] = 240*25 # simulate 4 serial transfers of 24h each
serial_params.all_params['batchDilution'] = True
serial_params.all_params['dilFactor'] = 0.01
serial_params.all_params['dilTime'] = 24

We run the simulation 

In [9]:
serial_expt = c.comets(world, serial_params)
serial_expt.run()


Running COMETS simulation ...
Done!


Now plot the biomasses of the two species during an experiment with 25 transfers every 24h 

In [10]:
biomass = serial_expt.total_biomass
biomass['transfer'] = biomass['cycle'] * comp_assay.parameters.all_params['timeStep']/24

myplot = biomass.drop(columns=['cycle', 't']).plot(x = 'transfer')

KeyError: "['t'] not found in axis"